In [10]:
import yfinance as yf
import pandas as pd
import numpy as np
import scipy.stats as sc
import scipy.linalg
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
sentiment = pd.read_csv("sentiment1.csv")
sentiment.head()

,TotalSentimentScore,Vaccine,Death,Shut,Date
0,0.085862,0,0,0,02/11/2020
1,-0.098021,2,3,1,02/12/2020
2,-0.093167,0,3,0,02/13/2020
3,0.009190,0,1,2,02/14/2020
4,0.000000,1,0,0,02/15/2020


In [90]:
delta = yf.Ticker("DAL").history(start="2020-02-11", end="2020-10-30")
delta

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-02-11,57.701254,58.694391,57.572149,58.257412,4566500,0.0,0
2020-02-12,58.883090,59.508764,58.773845,59.061855,5227500,0.0,0
2020-02-13,58.346797,58.803639,57.949540,58.724190,4353100,0.0,0
2020-02-14,58.833432,59.061853,58.177962,58.495766,3810100,0.0,0
2020-02-18,58.465970,59.061854,58.009128,58.317001,5690600,0.0,0
...,...,...,...,...,...,...,...
2020-10-23,33.520000,34.040001,32.900002,34.000000,15247300,0.0,0
2020-10-26,33.500000,33.500000,31.530001,31.930000,16558300,0.0,0
2020-10-27,31.750000,32.060001,30.690001,30.709999,13442000,0.0,0


In [92]:
dates =[]
for x in range(len(delta)):
    newdate = str(delta.index[x])
    newdate = newdate[0:10]
    dates.append(newdate)
delta['Date'] = dates
delta

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Date
Date,,,,,,,,
2020-02-11,57.701254,58.694391,57.572149,58.257412,4566500,0.0,0,2020-02-11
2020-02-12,58.883090,59.508764,58.773845,59.061855,5227500,0.0,0,2020-02-12
2020-02-13,58.346797,58.803639,57.949540,58.724190,4353100,0.0,0,2020-02-13
2020-02-14,58.833432,59.061853,58.177962,58.495766,3810100,0.0,0,2020-02-14
2020-02-18,58.465970,59.061854,58.009128,58.317001,5690600,0.0,0,2020-02-18
...,...,...,...,...,...,...,...,...
2020-10-23,33.520000,34.040001,32.900002,34.000000,15247300,0.0,0,2020-10-23
2020-10-26,33.500000,33.500000,31.530001,31.930000,16558300,0.0,0,2020-10-26
2020-10-27,31.750000,32.060001,30.690001,30.709999,13442000,0.0,0,2020-10-27


In [99]:
price = np.zeros(263)
i = 0
for date in sentiment.Date:
    if date in delta.Date:
        d = date[6:] + "-" + date[0:2] + "-" + date[3:5]  
        price[i] = delta[delta.Date == d].Close[0]
    else:
        price[i] = 0
    i+=1
for i in range(1, len(price)):
    if price[i] == 0 :
        if i != len(price)-1:
            price[i] = .5*(price[i-1]+price[i+1])
        else:
            price[i] = price[i-1]
print(price)

[58.25741196 59.06185532 58.72418976 58.49576569 29.24788284 14.62394142
 36.47047138 58.31700134 58.50999832 58.38000107 57.86999893 28.93499947
 41.5824995  54.22999954 50.88999939 49.59000015 48.18999863 46.13000107
 23.06500053 35.11250019 47.15999985 46.18000031 48.5        45.00999832
 45.88999939 22.94499969 33.23250008 43.52000046 45.47000122 42.66999817
 33.70999908 38.36000061 19.18000031 27.49500084 35.81000137 31.73999977
 23.48999977 21.51000023 21.35000038 10.67500019 16.44749975 22.21999931
 26.88999939 31.10000038 31.70000076 29.54999924 14.77499962 21.72249985
 28.67000008 28.53000069 23.87000084 22.68000031 22.47999954 11.23999977
 16.77999973 22.31999969 22.25       23.22999954 24.38999939 12.19499969
  6.09749985 14.67374992 23.25       24.54000092 24.35000038 22.78000069
 24.27000046 12.13500023 17.88749981 23.63999939 23.10000038 22.46999931
 22.47999954 22.40999985 11.20499992 16.68249989 22.15999985 24.34000015
 27.31999969 25.90999985 24.12000084 12.06000042 17

In [100]:
df = sentiment
df["Price"] = price
df

,TotalSentimentScore,Vaccine,Death,Shut,Date,Price
0,0.085862,0,0,0,02/11/2020,58.257412
1,-0.098021,2,3,1,02/12/2020,59.061855
2,-0.093167,0,3,0,02/13/2020,58.724190
3,0.009190,0,1,2,02/14/2020,58.495766
4,0.000000,1,0,0,02/15/2020,29.247883
...,...,...,...,...,...,...
258,-0.045096,6,4,3,10/26/2020,31.930000
259,-0.109511,5,8,2,10/27/2020,30.709999
260,-0.030883,4,4,8,10/28/2020,29.650000
261,-0.052484,3,1,6,10/29/2020,30.760000
